#### To run the script, select _Run_ > _Run All Cells_ from the main toolbar. 
This notebook is fully editable within the browser.

In [ ]:
# import libraries
import geopandas as gpd
import folium
import branca
import time

# URL to source file from ich-tanke-strom.ch project (data in EPSG:4326)
source_json = "https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/ch.bfe.ladestellen-elektromobilitaet_fr.json"  # can be updated #

# load data 
lausanne = gpd.read_file("resources/limites_lausanne_2056.geojson") 
bornes = gpd.read_file(source_json).to_crs(lausanne.crs) # ensure both datasets share the same CRS

# spatial join to identify charging stations within lausanne
bornesLS = gpd.sjoin(bornes, lausanne, how='inner')

# time stamp
now = time.strftime("%Y-%m-%d %H:%M:%S %Z", time.localtime())

# initialise map
fig = branca.element.Figure(height=600)    
m = folium.Map([46.519630, 6.632130], tiles="cartodbpositron", zoom_start=12)
fig.add_child(m)

# create groups for layer control
available = folium.FeatureGroup(name="Available", show=True).add_to(m)
unavailable = folium.FeatureGroup(name="Unavailable/Unknown", show=True).add_to(m)

# define function for marker colours
def colours(feature):
    if feature['properties']["Availability"] == "Available":
        return "green"
    if feature['properties']["Availability"] == "Occupied":
        return "red" 
    if feature['properties']["Availability"] == "Out of service":
        return "black"
    if feature['properties']["Availability"] == "Unknown":
        return "gray"
    else:
        return "gray"

# add objects to map
folium.GeoJson(bornesLS[bornesLS['Availability'] == "Available"],
               name = "Bornes de recharge",
               marker=folium.Marker(icon=folium.Icon(icon='glyphicon-flash')),
               tooltip=folium.GeoJsonTooltip(fields=["Availability"], aliases=["Status:"]),
               style_function=lambda feature: {
                   'markerColor': colours(feature)}).add_to(available)
folium.GeoJson(bornesLS[bornesLS['Availability'] != "Available"],
               name = "Bornes de recharge",
               marker=folium.Marker(icon=folium.Icon(icon='glyphicon-flash')),
               tooltip=folium.GeoJsonTooltip(fields=["Availability"], aliases=["Status:"]),
               style_function=lambda feature: {
                   'markerColor': colours(feature)}).add_to(unavailable)
folium.GeoJson(lausanne,
               name = "Ville de Lausanne",
               style_function=lambda feature: {
                   "fillColor": "darkgray",
                   "fillOpacity": 0.5,
                   "color": "darkgray",
                   "weight": 2},
               show = True).add_to(m)
folium.TileLayer("OpenStreetMap", show = False).add_to(m)
folium.LayerControl().add_to(m)

# print results and map
print("# Available:", len(bornesLS[bornesLS['Availability']== "Available"]))
print("# Occupied:", len(bornesLS[bornesLS['Availability']== "Occupied"]))
print("# Out of service:", len(bornesLS[bornesLS['Availability']== "Out of service"]))
print("# Unknown:", len(bornesLS[bornesLS['Availability']== "Unknown"]))
print("TOTAL:", len(bornesLS))
print(f"({now})")
m